In [62]:
%pip install sentence_transformers
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [63]:
df = pd.read_csv("../data/cleaned_job_descriptions.csv")
df.shape

(3449, 7)

In [64]:
#confirm that the number of lines is equal to the number of rows in the dataframe
# df = df.drop_duplicates()
# df['Role'].unique()
# roles_to_keep = ['Frontend Web Developer', 'Wireless Network Engineer', 'User Interface Designer', 'Front-End Developer', 'IT Support Specialist', 'Database Developer', 'Social Science Researcher', 'Record Keeper',
#        'Staff Nurse', 'International Tax Consultant', 'Risk Analyst',
#        'Speech Pathologist', 'Regional Sales Director',
#        'Mental Health Counselor', 'Urban Planner',
#        'Strategic Account Manager', 'User Researcher',
#        'Financial Analyst', 'Desktop Support Technician' ]

# df = df[df['Role'].isin(roles_to_keep)]
# df['Role'].unique()
# df.shape

# df

#(1610462, 7)
#shape after filtering to keep only the roles we are interested in
#(86322, 7)
lines = []
for index, row in df.iterrows():
    row_string = " ".join(row.astype(str))  # Convert each value to string and join
    lines.append(row_string)

print(lines[0])


1 to 15 Years BBA Yaounde Database Administrator Database Developer A Database Developer designs, develops, and maintains databases. They create efficient data structures, write queries, and ensure data security and integrity. News Corp.


In [65]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
embeddings = model.encode(lines, show_progress_bar=True)
embeddings.shape

Batches:   0%|          | 0/108 [00:00<?, ?it/s]

(3449, 384)

In [66]:
d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embeddings)
index.is_trained
print(type(index))


import joblib
#move the model and the index to the model folder
# print("type of model", type(model))
# print("type of index", type(index))

# #save using faiss
# faiss.write_index(index, "../model/faiss_index.faiss")
# #save using joblib
# joblib.dump(model, "../model/model.joblib")


<class 'faiss.swigfaiss.IndexFlatL2'>


In [67]:
k = 4
query = "I am a student looking for a job in the tech industry, I want to do cyber security"
query_embedding = model.encode(query)
query_embedding = query_embedding.reshape(1, -1)

In [68]:
D, I = index.search(query_embedding, k)
indices = I[0]
upper_bound = df.shape[0]

for index1 in indices:
    if index1 < upper_bound:
        print(df.iloc[index1]['Job_Title'], '\n\n')
    else:
        print("No match found")

Database Administrator 


Database Administrator 


Database Administrator 


Database Administrator 




In [69]:

print("type of model", type(model))
print("type of index", type(index))

#save using faiss
faiss.write_index(index, "../model/faiss_index.faiss")
#save using joblib
# joblib.dump(model, "../model/model.joblib")





type of model <class 'sentence_transformers.SentenceTransformer.SentenceTransformer'>
type of index <class 'faiss.swigfaiss.IndexFlatL2'>


In [70]:
#save the dataframe as well
df.to_pickle("../model/df.pkl")

In [71]:
#Save the model
joblib.dump(model, "../model/model.joblib")


['../model/model.joblib']

In [72]:
import os


file_size = os.path.getsize("../model/df.pkl")
model_size = os.path.getsize("../model/model.joblib")   
index_size = os.path.getsize("../model/faiss_index.faiss")
# joblib.dump(model, "../model/model.joblib")


# Print the file size in bytes, and convert to KB or MB if needed
print(f"File size: {file_size} bytes")
print(f"File size: {file_size / 1024} KB")
print(f"File size: {file_size / (1024 * 1024)} MB")

print(f"Model size: {model_size} bytes")
print(f"Model size: {model_size / 1024} KB")
print(f"Model size: {model_size / (1024 * 1024)} MB")


print(f"Index size: {index_size} bytes")
print(f"Index size: {index_size / 1024} KB")
print(f"Index size: {index_size / (1024 * 1024)} MB")

File size: 107008 bytes
File size: 104.5 KB
File size: 0.10205078125 MB
Model size: 91387369 bytes
Model size: 89245.4775390625 KB
Model size: 87.15378665924072 MB
Index size: 5297709 bytes
Index size: 5173.5439453125 KB
Index size: 5.052289009094238 MB
